# Peer-graded Assignment: Эксперименты с моделью

На прошлой неделе вы поучаствовали в соревновании на kaggle и, наверняка, большинство успешно справилось с прохождением baseline, а значит пора двигаться дальше - заняться оптимизацией модели, провести серию экспериментов и построить сильное финальное решения.

В этом задании вам нужно провести ряд эскпериментов, оценить качество полученных в процессе экспериментирования моделей и выбрать лучшее решение. 

Задание будет оцениваться на основании загруженного jupyther notebook и развернутых ответов на поставленные вопросы.

### Предобработка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, learning_curve, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score

In [2]:
data = pd.read_csv("orange_small_churn_train_data.csv")
data_test = pd.read_csv("orange_small_churn_test_data.csv")
data.drop("ID", axis = 1, inplace = True)
data_test.drop("ID", axis = 1, inplace = True)
print data.shape, data_test.shape

(40000, 231) (10000, 230)


In [3]:
num_features = list(data.columns[:190])
cat_features = list(data.columns[190:230])

Удалим признаки с большим количеством пропусков.

In [4]:
empty_features = []
c = 0.3
for feat in data.columns:
    nulls = data[feat].isnull().value_counts()
    try:
        not_nulls = nulls[False]
        if not_nulls < c*40000:
            empty_features.append(feat)
    except:    
        empty_features.append(feat)
print "number of empty features is", len(empty_features)

number of empty features is 156


In [5]:
for feat in empty_features:
    #data.drop(feat, axis = 1, inplace = True)    
    if feat in num_features:
        num_features.remove(feat)
    else:
        cat_features.remove(feat)
    data.drop(feat,axis=1, inplace=True)
    data_test.drop(feat,axis=1, inplace=True)

In [6]:
print data.shape, data_test.shape

(40000, 75) (10000, 74)


Заменим пропущенные значения числовых признаков на максимальное значение + 1

In [7]:
maxs = data.max(axis = 0)

In [8]:
for i,feat in enumerate(num_features):
    fill_value = maxs[i]+1
    data.fillna({feat: fill_value}, inplace=True)
    data_test.fillna({feat: fill_value}, inplace=True)


Заменим значения категориальных признаков на частоты

In [9]:
data_cat_all = data[cat_features].append(data_test[cat_features])

for feat in data_cat_all.columns:
    data_cat_all[feat] = data_cat_all[feat].map(data_cat_all.groupby(feat).size())
    
data[cat_features] = data_cat_all.loc[:39999,:]
data_test[cat_features] = data_cat_all.loc[39999:,:]

Оставшиеся пропущенные значения категориальных признаков заменим нулями.

In [10]:
data.fillna(0., inplace=True)
data_test.fillna(0., inplace=True)

In [11]:
print data.shape
print data_test.shape
data.head()

(40000, 75)
(10000, 74)


,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229,labels
0,3052.0,36.0,97365.0,480.0,600.0,20.0,480.0,200.00,0.0,82752.0,...,3,1662,3,36608.0,0.0,2614,2342,1477,0.0,-1
1,1813.0,7.0,636.0,212.0,265.0,2.0,128.0,166.56,0.0,2706120.0,...,1,37009,1,36608.0,11072.0,2108,35156,4354,9804.0,-1
2,1953.0,7.0,448.0,176.0,220.0,0.0,72.0,311.76,0.0,4698780.0,...,4441,6199,4441,36608.0,0.0,8031,6153,2672,9804.0,-1
3,1533.0,7.0,4.0,332.0,415.0,0.0,144.0,220.08,5.0,864384.0,...,34,37009,34,36608.0,0.0,2108,35156,32703,0.0,1
4,686.0,7.0,0.0,160.0,200.0,2.0,48.0,278.00,0.0,4364880.0,...,2,37009,2,36608.0,0.0,4176,35156,32703,0.0,-1


## Инструкции

#### Размер выборки

1\. Начнем с простого. Давайте оценим как много объектов действительно нужно для построения качественной модели. Для обучения доступна достаточно большая выборка и может так оказаться, что начиная с некоторого момента рост размера обучающей выборки перестает влиять на качество модели. Постройте кривые обучения, обучая модель на выборках разного размера начиная с небольшого количество объектов в обучающей выборке и постепенно наращивая её размер с некоторым шагом. Обратите внимание на `sklearn.model_selection.learning_curve`

In [12]:
clf = GradientBoostingClassifier(random_state = 123)
X = data.drop("labels", axis = 1)
y = data['labels']

In [14]:
train_sizes = np.arange(0.1,1.1,0.1)
train_sizes

array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [15]:
lc = learning_curve(clf, X, y, train_sizes = train_sizes, cv =5, scoring = 'roc_auc', 
                    shuffle = True, random_state = 12  )

In [16]:
size = lc[0]
train_score = lc[1]
test_score = lc[2]

for s, t1, t2 in zip(size,train_score, test_score):
    print 'for sample of %d auc roc on train is %f, on test is %f'%(s, np.mean(t1), np.mean(t2))

for sample of 3199 auc roc on train is 0.960695, on test is 0.706019
for sample of 6399 auc roc on train is 0.898029, on test is 0.723727
for sample of 9599 auc roc on train is 0.863657, on test is 0.730193
for sample of 12799 auc roc on train is 0.842792, on test is 0.736004
for sample of 15999 auc roc on train is 0.828983, on test is 0.734760
for sample of 19199 auc roc on train is 0.817401, on test is 0.738203
for sample of 22399 auc roc on train is 0.809868, on test is 0.739753
for sample of 25599 auc roc on train is 0.804365, on test is 0.741986
for sample of 28799 auc roc on train is 0.798450, on test is 0.741976
for sample of 31999 auc roc on train is 0.794628, on test is 0.744129


Качество на тестовой выборке растет с увеличением выборки. Для лучшего качества стоит ипользовать для обучения 80% выборки и более.

#### Балансировка выборки

2\. Часто несбалансированные по классам выборки приводят к различным проблемам при обучении моделей. Давайте попробуем по-разному обработать выборку, поиграть с распределением объектов по классам и сделать выводы о том, как соотношение классов влияет на качество модели.

2.1\. Задайте веса объектам так, чтобы соотношение классов с учетом весов объектов изменилось. Попробуйте не менее трёх различных вариантов весов. Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

In [14]:
churn_class = data[data["labels"] == 1]
stay_class = data[data["labels"] == -1]
churn_class.shape

(2976, 75)

In [75]:
X = data.drop('labels', axis = 1)
y = data['labels']
roc_auc = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for weight [1,1] is', np.mean(roc_auc)

roc auc for weight [1,1] is 0.744002126011


In [15]:
data_21 = data.append(churn_class, ignore_index = True)
data_21.shape

(42976, 75)

In [98]:
X = data_21.drop('labels', axis = 1)
y = data_21['labels']
roc_auc21 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for weight [2,1] is', np.mean(roc_auc21)

roc auc for weight [2,1] is 0.770767809306


In [16]:
data_41 = data_21.append(churn_class, ignore_index = True)
data_41 = data_41.append(churn_class, ignore_index = True)
data_41.shape

(48928, 75)

In [101]:
X = data_41.drop('labels', axis = 1)
y = data_41['labels']
roc_auc41 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for weight [4,1] is', np.mean(roc_auc41)

roc auc for weight [4,1] is 0.784580784503


In [17]:
data_61 = data_41.append(churn_class, ignore_index = True)
data_61 = data_61.append(churn_class, ignore_index = True)
data_61.shape

(54880, 75)

In [103]:
X = data_61.drop('labels', axis = 1)
y = data_61['labels']
roc_auc61 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for weight [6,1] is', np.mean(roc_auc61)

roc auc for weight [6,1] is 0.785884211082


In [104]:
data_12 = data.append(stay_class, ignore_index = True)
data_12.shape

(77024, 75)

In [105]:
X = data_12.drop('labels', axis = 1)
y = data_12['labels']
roc_auc12 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for weight [1,2] is', np.mean(roc_auc12)

roc auc for weight [1,2] is 0.748324819344


In [106]:
del data_12, data_21, data_41

Увеличение веса класса "отток" дает улучшение качества. Лучший результат у веса [6,1] для классов отток и не отток соответственно.

2.2\. Примените к выборке технологию undersampling: для этого нужно убрать из обучения некоторое количество объектов большего класса таким образом, чтобы соотношение классов изменилось. Попробуйте не менее трёх различных вариантов undersampling (варианты могут отличаться как по количество отфильтрованных объектов, так и по принципу выборка объектов для отсеивания из выборки). Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

In [22]:
churn_class = data[data["labels"] == 1]
stay_class = data[data["labels"] == -1]
churn_class.shape

(2976, 75)

In [63]:
#churn to stay ratio is about 1:12
sample_size = len(churn_class)
random_indices1 = np.random.choice(stay_class.index, sample_size*3, replace=False)
random_indices2 = np.random.choice(stay_class.index, sample_size*3, replace=True)
random_indices3 = np.random.choice(stay_class.index, sample_size*2, replace=True)

In [62]:
row_indexer = list(churn_class.index) + list(random_indices1)
X = data.iloc[row_indexer, :74]
y = data.iloc[row_indexer,74]
roc_auc1 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for undersampling (replace = False), weight [1,0.25], sample size %d is %f'%(len(row_indexer),np.mean(roc_auc1))

roc auc for undersampling (replace = False), weight [1,0.25], sample size 11904 is 0.739604


In [61]:
row_indexer = list(churn_class.index) + list(random_indices2)
X = data.iloc[row_indexer, :74]
y = data.iloc[row_indexer,74]
roc_auc2 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for undersampling (replace = True), weight [1,0.25], sample size %d is %f'%(len(row_indexer),np.mean(roc_auc2))

roc auc for undersampling (replace = True), weight [1,0.25], sample size 11904 is 0.749055


In [67]:
row_indexer = list(churn_class.index) + list(random_indices3)
X = data.iloc[row_indexer, :74]
y = data.iloc[row_indexer,74]
roc_auc3 = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc for undersampling (replace = True), weight [1,0.16], sample size %d is %f'%(len(row_indexer),np.mean(roc_auc3))

roc auc for undersampling (replace = True), weight [1,0.16], sample size 8928 is 0.747624


Результаты хуже, чем у оверсемплинга, что неудивительно, так как часть информации остается за бортом. Вариант с возвращениями работает лучше.

#### Обработка числовых признаков

3\. Теперь перейдем к работе с признаками. Ранее вы реализовали несколько стратегий для обработки пропущенных значений. Сравните эти стратегии между собой с помощью оценки качества моделей кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка пропущенных значений сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

Пропущенные значения для числовых признаков обрабатывались так:
- заполнялись нулями
- заполнялись средними
- заполнялись медианами
- заполнялись минимальным значением - 1 (чтобы деревья могли отсортировывать пропущенные значения одной вершиной)
- заполнялись максимальным значением + 1 (то же)

Больших различий обнаружено не было, только выяснилось что заполнять медианами хуже всего, а лучший результат дает последний пункт (но все равно это улучшения порядка нескольких тысячных для auc_roc) 

In [106]:
data3 = pd.read_csv("orange_small_churn_train_data.csv")
data3.drop("ID", axis = 1, inplace = True)
print data3.shape

(40000, 231)


In [107]:
for feat in empty_features:
    data3.drop(feat,axis=1, inplace=True)
print data3.shape

(40000, 75)


In [108]:
mins = data3.min(axis = 0)
maxs = data3.max(axis = 0)

In [87]:
data30 = data3.fillna(0)

In [130]:
for i, feat in enumerate(num_features):
    m1 = np.mean(data3[feat].dropna())
    m2 = np.median(data3[feat].dropna())
    m3 = mins[i]-1
    m4 = maxs[i]+1
    m_list = [m1, m2, m3, m4]
    #print feat, m_list
    if i == 0:
        data31 = data3.fillna({feat: m1})
        data32 = data3.fillna({feat: m2})
        data33 = data3.fillna({feat: m3})
        data34 = data3.fillna({feat: m4})
        data_list = [data31, data32, data33, data34]
    else: 
        for d, m in zip(data_list, m_list):
            d = d.fillna({feat:m}, inplace = True)


In [131]:
data_list = [data30, data31, data32, data33, data34]
descr_list = ['zeros', 'means', 'meds', 'min - 1', 'max + 1']
roc_auc_list = []
for d, descr in zip(data_list, descr_list):
    X = d[num_features].join(data[cat_features])
    y = d['labels']
    ra = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
    roc_auc_list.append(ra)
    print 'for %s filling roc_auc is %f'%(descr, np.mean(ra))

for zeros filling roc_auc is 0.742215
for means filling roc_auc is 0.742754
for meds filling roc_auc is 0.734380
for min - 1 filling roc_auc is 0.740390
for max + 1 filling roc_auc is 0.744002


#### Обработка категоральных признаков

4\. Также вы уже реализовали несколько стратегий для обработки категориальных признаков. Сравните эти стратегии между собой с помощью оценки качества моделей по кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка категориальных признаков сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

Категориальные признаки обрабатывались так:
- хэширование признаков
- OneHotEncoding (для признаков с <400 значений) + хэширование (остальных)
- заполнение частотой данного значени

Лучший результат показал последний вариант. Второй вариант неудобен тем, что добавляет большое количество новых признаков, что замедляет работу бустинга, и вообще он больше подходит для регрессий, чем для деревьев.

In [264]:
data4 = pd.read_csv("orange_small_churn_train_data.csv")
data4.drop("ID", axis = 1, inplace = True)
print data4.shape
for feat in empty_features:
    data4.drop(feat,axis=1, inplace=True)
print data4.shape

(40000, 231)
(40000, 75)


In [266]:
data4[num_features] = data[num_features]

In [267]:
data4.fillna(0,inplace = True)

In [268]:
from sklearn.feature_extraction import FeatureHasher
hash_space = 400

cat_x_hashed = pd.DataFrame()
data_set = [data4]
hash_set = [cat_x_hashed]

for feat in cat_features:
    for d, h in zip(data_set, hash_set):
        feat_hashed = [hash(x) % hash_space for x in d[feat]]
        h[str(feat)] = pd.Series(feat_hashed)

In [269]:
cat_x_hashed.head()

,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var200,Var202,Var203,...,Var219,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229
0,175,61,366,320,266,147,357,0,191,154,...,167,225,332,132,206,0,313,184,60,0
1,240,249,366,320,348,10,111,33,220,154,...,167,34,189,101,206,368,277,0,386,207
2,113,98,366,320,192,151,44,0,3,154,...,167,71,102,248,206,0,304,237,314,207
3,67,120,366,320,387,58,252,0,99,156,...,167,237,189,216,206,0,277,0,113,0
4,198,120,366,320,301,76,353,0,257,154,...,167,289,189,226,206,0,128,0,113,0


5\. Все ли признаки оказались полезными для построения моделей? Проведите процедуру отбора признаков, попробуйте разные варианты отбора (обратите внимание на модуль `sklearn.feature_selection`). Например, можно выбрасывать случайные признаки или строить отбор на основе l1-регуляризации - отфильтровать из обучения признаки, которые получат нулевой вес при построении регрессии с l1-регуляризацией (`sklearn.linear_model.Lasso`). И всегда можно придумать что-то своё=) Попробуйте как минимум 2 различные стратегии, сравните результаты. Помог ли отбор признаков улучшить качество модели? Поясните свой ответ.

Уже используется отбор признаков по количеству пропущенных значений (выброшены те, у которых количество пропусков более 80%). Это несильно улучшает результат, зато помогает быстрее обсчитывать модели. Попробуем дальнейший отбор признаков по дисперсии значений.

In [73]:
from sklearn.feature_selection import VarianceThreshold

In [72]:
X = data.drop('labels', axis = 1)
y = data['labels']

In [96]:
#roc_auc without variance selection is 0.744002126011
selector = VarianceThreshold(500.0)
X_sel = selector.fit_transform(X)
roc_auc = cross_val_score(clf, X_sel, y, cv=5, scoring='roc_auc')
print 'roc auc for %d selected features via VarieanceThreshold is %f'% (X_sel.shape[1], np.mean(roc_auc))

roc auc for 64 selected features via VarieanceThreshold is 0.743889


In [94]:
selector = VarianceThreshold(1000.0)
X_sel = selector.fit_transform(X)
roc_auc = cross_val_score(clf, X_sel, y, cv=5, scoring='roc_auc')
print 'roc auc for %d selected features via VarieanceThreshold is %f'% (X_sel.shape[1], np.mean(roc_auc))

roc auc for 62 selected features via VarieanceThreshold is 0.745222


In [107]:
selector = VarianceThreshold(1500.0)
X_sel = selector.fit_transform(X)
roc_auc = cross_val_score(clf, X_sel, y, cv=5, scoring='roc_auc')
print 'roc auc for %d selected features via VarieanceThreshold is %f'% (X_sel.shape[1], np.mean(roc_auc))

roc auc for 60 selected features via VarieanceThreshold is 0.697783


Как видно, отбор признаков по дисперсии для значений 500 и 1000 значимого влияния на результат не оказывает, а при значении 1500 качество падает -- мы начинаем отсеивать важные признаки.

6\. Подберите оптимальные параметры модели. Обратите внимание, что в зависимости от того, как вы обработали исходные данные, сделали ли балансировку классов, сколько объектов оставили в обучающей выборке и др. оптимальные значения параметров могут меняться. Возьмите наилучшее из ваших решений на текущий момент и проведите процедуру подбора параметров модели (обратите внимание на `sklearn.model_selection.GridSearchCV`) Как подбор параметров повлиял на качество модели?

Для поиска параметров по сетке будем использовать лучшую на данный момент конфигурацию: балансировка с весами [6,1], заполнение пропущенных значений вещественных признаков на max + 1, кодирование категориальных признаков частотами, удалений пустых и почти пустых (>80% Nans) признаков.

In [261]:
X = data_61.drop('labels', axis = 1)
y = data_61['labels']
skf = StratifiedKFold(n_splits=5, shuffle=True)
a = skf.split(X, y)
for train_index, test_index in skf.split(X, y):    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index] 
print X_train.shape, X_test.shape

(43905, 74) (10975, 74)


In [271]:
from sklearn.model_selection import GridSearchCV
clf = GradientBoostingClassifier(random_state = 123)

params = {'n_estimators' : [50, 100, 200, 400],
    'max_depth' : [3,4], 
    'learning_rate' : [0.1, 0.03]
         }
gs1 = GridSearchCV(clf, params, cv = 5, scoring = 'roc_auc')
gs1.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=123, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 200, 400], 'learning_rate': [0.1, 0.03], 'max_depth': [3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [273]:
print gs1.best_score_
print gs1.best_params_

0.91991957633
{'n_estimators': 400, 'learning_rate': 0.1, 'max_depth': 4}


In [275]:
results = pd.DataFrame(gs1.cv_results_)
results

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_learning_rate,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,11.419707,0.031318,0.766616,0.772149,0.1,3,50,"{u'n_estimators': 50, u'learning_rate': 0.1, u...",13,0.766545,...,0.765806,0.772359,0.768223,0.772408,0.766347,0.771028,0.312612,0.001465,0.000839,0.000717
1,21.925349,0.043650,0.787367,0.796412,0.1,3,100,"{u'n_estimators': 100, u'learning_rate': 0.1, ...",10,0.788340,...,0.787694,0.796503,0.786925,0.795103,0.788316,0.797748,0.166803,0.001782,0.001041,0.000839
2,45.413946,0.074100,0.815060,0.829637,0.1,3,200,"{u'n_estimators': 200, u'learning_rate': 0.1, ...",6,0.816816,...,0.816156,0.829714,0.814510,0.829159,0.815148,0.829169,1.815741,0.003565,0.001435,0.000507
3,101.976458,0.139926,0.854798,0.876918,0.1,3,400,"{u'n_estimators': 400, u'learning_rate': 0.1, ...",3,0.856776,...,0.855523,0.876754,0.853544,0.875761,0.855393,0.877050,9.854533,0.008013,0.001452,0.000744
4,23.867842,0.043753,0.791351,0.801742,0.1,4,50,"{u'n_estimators': 50, u'learning_rate': 0.1, u...",9,0.792009,...,0.791861,0.803931,0.792511,0.801421,0.791621,0.802173,1.608810,0.005391,0.001331,0.001244
5,34.789507,0.050793,0.822030,0.837920,0.1,4,100,"{u'n_estimators': 100, u'learning_rate': 0.1, ...",5,0.823516,...,0.820693,0.837320,0.822295,0.837259,0.823300,0.838991,0.398701,0.001930,0.001305,0.000780
6,67.605288,0.081928,0.865966,0.889225,0.1,4,200,"{u'n_estimators': 200, u'learning_rate': 0.1, ...",2,0.870038,...,0.865091,0.888291,0.864389,0.888055,0.866385,0.890194,0.345275,0.002008,0.002199,0.001502
7,133.357233,0.144926,0.919920,0.948371,0.1,4,400,"{u'n_estimators': 400, u'learning_rate': 0.1, ...",1,0.921626,...,0.919577,0.947693,0.921260,0.948846,0.919539,0.947703,0.751673,0.002791,0.001440,0.001285
8,11.577536,0.030798,0.741621,0.744680,0.03,3,50,"{u'n_estimators': 50, u'learning_rate': 0.03, ...",16,0.742775,...,0.740761,0.744243,0.742219,0.744061,0.744051,0.744652,0.036626,0.000133,0.001968,0.000555
9,22.437614,0.044219,0.755673,0.760057,0.03,3,100,"{u'n_estimators': 100, u'learning_rate': 0.03,...",15,0.756873,...,0.754085,0.759787,0.756173,0.759868,0.757803,0.760379,0.082004,0.000205,0.001659,0.000213


7\. Предложите методику оценки того, какие признаки внесли наибольший вклад в модель (например, это могут быть веса в случае регрессии, а также большое количество моделей реализуют метод `feature_importances_` - оценка важности признаков). На основе предложенной методики проанализируйте, какие признаки внесли больший вклад в модель, а какие меньший?

In [18]:
X = data_61.drop('labels', axis = 1)
y = data_61['labels']
skf = StratifiedKFold(n_splits=5, shuffle=True)
a = skf.split(X, y)
for train_index, test_index in skf.split(X, y):    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index] 
print X_train.shape, X_test.shape

(43905, 74) (10975, 74)


In [19]:
clf = GradientBoostingClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 4, random_state = 123)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test) 
y_prob = [x[1] for x in y_pred]
roc_auc_ho = roc_auc_score(y_test, y_prob)
roc_auc_ho

0.91020823093044534

In [20]:
f_importance = pd.DataFrame(clf.feature_importances_, columns = ['importance'])
f_importance['feature'] = X.columns
f_importance.sort_values(by=['importance'], ascending = False).head(10)

,importance,feature
24,0.061246,Var113
11,0.060648,Var57
28,0.045030,Var126
42,0.036044,Var192
48,0.034103,Var199
0,0.029631,Var6
27,0.028810,Var125
41,0.028711,Var189
21,0.027933,Var94
18,0.027515,Var81


In [21]:
print '# of features with importance > 0 is', f_importance[f_importance.importance > 0].shape[0]
print '# of features with importance > 0.005 is', f_importance[f_importance.importance > 0.005].shape[0]

# of features with importance > 0 is 74
# of features with importance > 0.005 is 48


8\. Напоследок давайте посмотрим на объекты. На каких объектах достигается наибольшая ошибка классификации? Есть ли межу этими объектами что-то общее? Видны ли какие-либо закономерности? Предположите, почему наибольшая ошибка достигается именно на этих объектах. В данном случае "наибольшую" ошибку можно понимать как отнесение объекта с чужому классу с большой долей уверенности (с высокой вероятностью).

In [193]:
diff = pd.DataFrame(y_test)
diff['pred'] = y_prob
y_pred = clf.predict(X_test)
diff['class'] = y_pred
diff = diff[diff['labels'] != diff['class']]

In [210]:
diff['error'] = np.where(diff['class'] == 1, diff.pred, 1-diff.pred)

/Users/kustarev/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [214]:
diff.sort_values(by=['error'], ascending = False).head(10)

,labels,pred,class,error
40332,1,0.053852,-1,0.946148
43308,1,0.053852,-1,0.946148
46205,1,0.062220,-1,0.937780
45167,1,0.069607,-1,0.930393
53071,1,0.072159,-1,0.927841
51263,1,0.079509,-1,0.920491
40313,1,0.085103,-1,0.914897
46265,1,0.085103,-1,0.914897
50467,1,0.086406,-1,0.913594
41539,1,0.086406,-1,0.913594


In [248]:
inq = diff.sort_values(by=['error'], ascending = False)
inq = inq[inq.index < 40000]
inq_index = inq.sort_values(by=['error'], ascending = False).index
d = data.iloc[inq_index]

9\. По итогам проведенных экспериментов постройте финальную решение - модель с наилучшим качеством. Укажите, какие преобразования данных, параметры и пр. вы выбрали для построения финальной модели.

Для поиска параметров по сетке будем использовать лучшую на данный момент конфигурацию: балансировка с весами [6,1], заполнение пропущенных значений вещественных признаков на max + 1, кодирование категориальных признаков частотами, удалений пустых и почти пустых (>80% Nans) признаков. Параметры берем из research grid n_estimators = 400, learning_rate = 0.1, max_depth = 4.

In [277]:
clf = GradientBoostingClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 4, random_state = 123)
#clf = GradientBoostingClassifier(random_state = 123)

In [279]:
X = data_61.drop('labels', axis = 1)
y = data_61['labels']
skf = StratifiedKFold(n_splits=5, shuffle=True)
a = skf.split(X, y)
for train_index, test_index in skf.split(X, y):    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index] 
print X_train.shape, X_test.shape

0.91991957633


In [26]:
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test) 
y_prob = [x[1] for x in y_pred]
roc_auc_ho = roc_auc_score(y_test, y_prob)
#0.78026723279290344 with var selection
roc_auc_ho

0.91020823093044534

In [28]:
X = data.drop("labels", axis = 1)
y = data['labels']

roc_auc = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print 'roc auc best model in cv is %f'% (np.mean(roc_auc))

roc auc best model in cv is 0.732266


In [ ]:
### данное решение без оверсемплинга набрало на kaggle pub score 0.70311, pr score 0.73653
### с оверсемплингом [6,1] 0.72923, 0.68874


10\. Подумайте, можно ли еще улучшить модель? Что для этого можно сделать? 

Можно использовать другие классификаторы (например, ридж регрессию) и построить ансамбль из них.